In [37]:
# Script to combine multiple seasons df into one df

import pandas as pd
import os
from glob import glob

# Step 1: Point to the folder with all your CSVs
data_folder = "../data"
csv_files = sorted(glob(os.path.join(data_folder, "*_full.csv")))

# Step 2: Combine all CSVs
all_data = []
for file in csv_files:
    df = pd.read_csv(file)
    df['source_file'] = os.path.basename(file)  # optional: track file of origin
    all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Step 3: Save the combined file
combined_df.to_csv("../data/all_seasons_combined.csv", index=False)

# Step 4: Show basic info
print(f"✅ Combined {len(csv_files)} files")
print(f"📊 Total matches: {len(combined_df)}")
print("📄 Saved as: ../data/all_seasons_combined.csv")

✅ Combined 21 files
📊 Total matches: 7902
📄 Saved as: ../data/all_seasons_combined.csv


In [38]:
# script to clean the data


# Load the full dataset
df = pd.read_csv("../data/all_seasons_combined.csv")

# ✅ 1. Drop 'date' and 'source_file'
df.drop(columns=["date", "source_file"], inplace=True, errors='ignore')

# ✅ 2. Convert odds to float
for col in ["h_odd", "d_odd", "a_odd"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# ✅ 3. Extract starting year of season
df['season'] = df['season'].str[:4]

# ✅ 4. Split match_name
df[['home_team', 'away_team']] = df['match_name'].str.split(" vs ", expand=True)

# ✅ 5. Split result into home/away score
df[['home_score', 'away_score']] = df['result'].str.split("-", expand=True).astype("Int64")

# ✅ 6. Determine winner
def get_winner(row):
    if pd.isnull(row['home_score']) or pd.isnull(row['away_score']):
        return "UNKNOWN"
    elif row['home_score'] > row['away_score']:
        return "HOME_TEAM"
    elif row['home_score'] < row['away_score']:
        return "AWAY_TEAM"
    else:
        return "DRAW"

df['winner'] = df.apply(get_winner, axis=1)

# ✅ 7. Save cleaned version
df.to_csv("../data/all_seasons_cleaned.csv", index=False)
print("✅ Cleaned dataset saved as: data/all_seasons_cleaned.csv")


✅ Cleaned dataset saved as: data/all_seasons_cleaned.csv


In [39]:
# script for more cleaning and feature engineering of data

# Load cleaned data from previous step
df = pd.read_csv("../data/all_seasons_cleaned.csv")

# ✅ Drop the 'result' column (no longer needed)
df.drop(columns=['result'], inplace=True, errors='ignore')

# ✅ Convert American odds to Decimal odds
def american_to_decimal(odd):
    if pd.isnull(odd):
        return None
    try:
        odd = float(odd)
        if odd > 0:
            return round(1 + odd / 100, 2)
        else:
            return round(1 + 100 / abs(odd), 2)
    except:
        return None

for col in ['h_odd', 'd_odd', 'a_odd']:
    df[col] = df[col].apply(american_to_decimal)

# Save updated DataFrame
df.to_csv("../data/all_seasons_ready.csv", index=False)

print("✅ Cleaned odds and removed result column.")
print("📄 Saved as: ../data/all_seasons_ready.csv")


OSError: [Errno 22] Invalid argument: '../data/all_seasons_ready.csv'

In [ ]:
import pandas as pd
import numpy as np

# If not already loaded
df = pd.read_csv("../data/all_seasons_ready.csv")  # If you are inside /notebooks folder
print("✅ Dataset loaded with shape:", df.shape)

# Basic helper to calculate points from a match
def get_points(home_goals, away_goals):
    if home_goals > away_goals:
        return 3, 0
    elif home_goals < away_goals:
        return 0, 3
    else:
        return 1, 1

# Helper to update form streaks (win/draw/loss)
def update_streak(result, streaks):
    if result == "W":
        return (streaks[0] + 1, 0, 0)  # (win, draw, loss)
    elif result == "D":
        return (0, streaks[1] + 1, 0)
    else:
        return (0, 0, streaks[2] + 1)


✅ Dataset loaded with shape: (7902, 10)


In [ ]:
# Initialize history trackers
team_history = {}

# Placeholder for final feature-engineered rows
engineered_rows = []


In [ ]:
# Create features across matches
for idx, row in df.iterrows():
    season = int(row['season'])
    home = row['home_team']
    away = row['away_team']

    # Initialize teams if not seen before
    for team in [home, away]:
        if team not in team_history:
            team_history[team] = {
                "matches": [],  # (season, goals_for, goals_against, points)
                "last_season_rank": None,
            }

    # Get last season ranks
    ht_ls_rank = team_history[home]["last_season_rank"] if team_history[home]["last_season_rank"] else 10  # Assume middle rank if missing
    at_ls_rank = team_history[away]["last_season_rank"] if team_history[away]["last_season_rank"] else 10

    # Calculate rolling features for home
    ht_recent = [m for m in team_history[home]["matches"] if m[0] == season][-5:]  # Last 5 matches this season
    if ht_recent:
        ht_l_points = sum(m[3] for m in ht_recent)
        ht_l_goals = sum(m[1] for m in ht_recent)
        ht_l_goals_sf = sum(m[2] for m in ht_recent)
        ht_l_wavg_points = sum(m[3]*(i+1) for i, m in enumerate(reversed(ht_recent))) / sum(i+1 for i in range(len(ht_recent)))
        ht_l_wavg_goals = sum(m[1]*(i+1) for i, m in enumerate(reversed(ht_recent))) / sum(i+1 for i in range(len(ht_recent)))
        ht_l_wavg_goals_sf = sum(m[2]*(i+1) for i, m in enumerate(reversed(ht_recent))) / sum(i+1 for i in range(len(ht_recent)))
    else:
        ht_l_points = ht_l_goals = ht_l_goals_sf = ht_l_wavg_points = ht_l_wavg_goals = ht_l_wavg_goals_sf = 0

    # Calculate rolling features for away
    at_recent = [m for m in team_history[away]["matches"] if m[0] == season][-5:]
    if at_recent:
        at_l_points = sum(m[3] for m in at_recent)
        at_l_goals = sum(m[1] for m in at_recent)
        at_l_goals_sf = sum(m[2] for m in at_recent)
        at_l_wavg_points = sum(m[3]*(i+1) for i, m in enumerate(reversed(at_recent))) / sum(i+1 for i in range(len(at_recent)))
        at_l_wavg_goals = sum(m[1]*(i+1) for i, m in enumerate(reversed(at_recent))) / sum(i+1 for i in range(len(at_recent)))
        at_l_wavg_goals_sf = sum(m[2]*(i+1) for i, m in enumerate(reversed(at_recent))) / sum(i+1 for i in range(len(at_recent)))
    else:
        at_l_points = at_l_goals = at_l_goals_sf = at_l_wavg_points = at_l_wavg_goals = at_l_wavg_goals_sf = 0

    engineered_rows.append({
        "season": season,
        "match_name": row['match_name'],
        "home_team": home,
        "away_team": away,
        "winner": row['winner'],
        "home_score": row['home_score'],
        "away_score": row['away_score'],
        "h_odd": row['h_odd'],
        "d_odd": row['d_odd'],
        "a_odd": row['a_odd'],

        # New engineered features:
        "ht_ls_rank": ht_ls_rank,
        "at_ls_rank": at_ls_rank,
        "ht_l_points": ht_l_points,
        "ht_l_goals": ht_l_goals,
        "ht_l_goals_sf": ht_l_goals_sf,
        "ht_l_wavg_points": ht_l_wavg_points,
        "ht_l_wavg_goals": ht_l_wavg_goals,
        "ht_l_wavg_goals_sf": ht_l_wavg_goals_sf,
        "at_l_points": at_l_points,
        "at_l_goals": at_l_goals,
        "at_l_goals_sf": at_l_goals_sf,
        "at_l_wavg_points": at_l_wavg_points,
        "at_l_wavg_goals": at_l_wavg_goals,
        "at_l_wavg_goals_sf": at_l_wavg_goals_sf,
    })

    # Update team histories with current match result
    home_goals = row['home_score']
    away_goals = row['away_score']

    # Calculate points
    if home_goals > away_goals:
        h_pts, a_pts = 3, 0
    elif home_goals < away_goals:
        h_pts, a_pts = 0, 3
    else:
        h_pts, a_pts = 1, 1

    team_history[home]["matches"].append((season, home_goals, away_goals, h_pts))
    team_history[away]["matches"].append((season, away_goals, home_goals, a_pts))

    # (Optional) If end of season, simulate assigning last season rank
    if idx+1 < len(df) and df.iloc[idx+1]['season'] != row['season']:
        # Sort teams by points for last season rank (very basic)
        points_table = [(team, sum(m[3] for m in history["matches"] if m[0]==season)) for team, history in team_history.items()]
        points_table.sort(key=lambda x: x[1], reverse=True)
        for rank, (team, _) in enumerate(points_table, 1):
            team_history[team]["last_season_rank"] = rank


In [ ]:
# Create final feature DataFrame
final_feature_df = pd.DataFrame(engineered_rows)
final_feature_df.head()

,season,match_name,home_team,away_team,winner,home_score,away_score,h_odd,d_odd,a_odd,...,ht_l_goals_sf,ht_l_wavg_points,ht_l_wavg_goals,ht_l_wavg_goals_sf,at_l_points,at_l_goals,at_l_goals_sf,at_l_wavg_points,at_l_wavg_goals,at_l_wavg_goals_sf
0,2004,Birmingham vs Arsenal,Birmingham,Arsenal,HOME_TEAM,2,1,4.01,3.40,1.80,...,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2004,Bolton vs Everton,Bolton,Everton,HOME_TEAM,3,2,1.85,3.34,3.98,...,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2004,Charlton vs Crystal Palace,Charlton,Crystal Palace,DRAW,2,2,2.64,3.45,2.34,...,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2004,Fulham vs Norwich,Fulham,Norwich,HOME_TEAM,6,0,2.66,3.43,2.36,...,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2004,Liverpool vs Aston Villa,Liverpool,Aston Villa,HOME_TEAM,2,1,1.63,3.41,5.33,...,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


In [ ]:
final_feature_df.to_csv('feature_df.csv')